In [1]:
import warnings
warnings.filterwarnings('ignore') 

In [2]:
import openai
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from trafilatura import fetch_url
from trafilatura import extract
from bs4 import BeautifulSoup as bs
from random import choice
from colorama import Fore, Back, Style
from datetime import datetime
from rich import print as rprint

In [3]:
from rich import pretty
pretty.install()

In [4]:
from actions import get_page_source, text_extract, search_info, get_links, decide_SearchorLinks
from session_manager import start_session, USER_AGENTS
from config import Config

In [5]:
config = json.load(open("/Users/leszekbukowski/openai/config.json", "r"))
openai.api_key = config["KEY"]

In [6]:
model = "gpt-3.5-turbo-16k-0613"
#model = "gpt-4-0614"

In [7]:
READ_PROMPTS = json.load(open("read_prompts.json", "r"))
SEARCH_PROMPTS = json.load(open("search_prompts.json", "r"))

In [8]:
MEMORIES = {"questions": [], "theses": []}

def structure_memory(raw_thesis: str, page_url: str):
    theses = [thesis[2:].strip() for thesis in raw_thesis.split("\n")]
    timed_theses = {page_url: [{"memory": thesis, "time": datetime.now().isoformat()[:-7]} for thesis in theses]}
    return timed_theses

In [9]:
functions = [
        {
            "name": "decide_SearchorLinks",
            "description": "Returns dict with decision.",
            "parameters": {
                "type": "object",
                "properties": {
                    "what_to_search": {
                        "type": "string",
                        "description": "Query to search for",
                    },
                    "link_num": {"type": "string", "description": "Link number to click"},
                },
                "required": ["what_to_search", "link_num"],
            },
        }
    ]

In [10]:
S = start_session(headless=True)
page_source = get_page_source(S, "https://plato.stanford.edu/entries/poincare/")
#page_source = get_page_source(S, "https://en.wikipedia.org/wiki/Warsaw")
page_url = S.current_url
text = text_extract(page_source)[:50000] 
print(Style.BRIGHT+Fore.BLUE + f"I am reading the following page: {page_url}", end="\n\n")
#links_to_follow = [l.get_attribute("href") for l in get_links(S) if l.get_attribute("href")]
links_to_follow = [(l.accessible_name, l.get_attribute("href")) for l in get_links(S) if l.get_attribute("href") and l.accessible_name]
if len(links_to_follow) > 100:
    links_to_follow = links_to_follow[:100]
texted_links_to_follow = "\n".join([f"{i+1}. {link[0]}, {link[1]}" for i, link in enumerate(links_to_follow)])

messages = [{"role": "system", "content": READ_PROMPTS["navigator_manifest"]},
            {"role": "user", "content": READ_PROMPTS["reflection"].format(page_url, text)}]


# Generates memories from the content of the page
raw_page_memories = openai.ChatCompletion.create(
    model=model,
    temperature=0.8,
    messages=messages
)

page_memories = raw_page_memories["choices"][0]["message"]["content"]
print(Style.BRIGHT + Fore.LIGHTGREEN_EX + page_memories, end="\n\n")
MEMORIES["theses"].append(structure_memory(page_memories, page_url))
messages.append({"role": "assistant", "content": page_memories})
messages.append({"role": "user", "content": READ_PROMPTS["autoreflection"].format(page_memories)})


# Generates the question to ask
generated_question = openai.ChatCompletion.create(
    model=model,
    temperature=0.8,
    messages=messages
)

current_question = generated_question["choices"][0]["message"]["content"]
print(Style.BRIGHT + Fore.YELLOW + current_question, end="\n\n")
MEMORIES["questions"].append({page_url: current_question})
messages.append({"role": "assistant", "content": current_question})
messages.append({"role": "user", "content": READ_PROMPTS["next_move_consideration"].format(current_question,
                                                                            texted_links_to_follow,
                                                                            Config.SERCH_ENGINE)})


# Make decision about the futher steps
filtered_messages = [messages[0], messages[-1]]
raw_decision = openai.ChatCompletion.create(
        model=model,
        temperature=0.8,
        messages=filtered_messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )

decision_message = raw_decision["choices"][0]["message"]
if decision_message.get("function_call"):
    #print(decision_message)
    decision = json.loads(decision_message["function_call"]["arguments"])
    if decision["link_num"]:
        print(Style.BRIGHT + Fore.LIGHTMAGENTA_EX +  f"follow link: {decision['link_num']}", end="\n\n")
    else:
        print(Style.BRIGHT + Fore.LIGHTMAGENTA_EX + f"Searching for: {decision['what_to_search']}", end="\n\n")
    
    search_phrase = decision["what_to_search"]


I am reading the following page: https://plato.stanford.edu/entries/poincare/

1. Henri Poincaré was a polymath, known for his contributions in mathematics, theoretical physics, and philosophy of science.

2. Poincaré's philosophy of science is influenced by the reflections of Ernst Mach, James Maxwell, and Hermann von Helmholtz, as well as the philosophical doctrines of his time.

3. Poincaré's philosophy of mathematics rejects the idea that logic alone is sufficient to express the essential structure of mathematical reasoning.

4. Poincaré's critique of logicism and formalism in mathematics led him to develop his own geometric conventionalism.

5. Poincaré's view of geometry is that it is a system of relations, not a study of inherent properties of objects, and he argues that different geometries can be equally valid.

6. Poincaré's concept of conventions in geometry suggests that the choice of axioms and definitions is not determined by the nature of objects, but rather by the conve

In [11]:
Ssearch = start_session(headless=True)
#search_source = get_page_source(Ssearch, Config.SERCH_ENGINE)
search_links, link_synopsis = search_info(Ssearch, search_phrase, S.current_url)


search_messages = [{"role": "system", "content": READ_PROMPTS["navigator_manifest"]},
            {"role": "user", "content": SEARCH_PROMPTS["search_consideration"].format(search_phrase, link_synopsis)}]

raw_search_link_num = openai.ChatCompletion.create(
    model=model,
    temperature=0.8,
    messages=search_messages
)

search_link_num = raw_search_link_num["choices"][0]["message"]["content"]
print(Style.BRIGHT + Fore.LIGHTYELLOW_EX + search_link_num, end="\n\n")

#Ssearch.quit()

link_num: 1



In [121]:
# options = Options()

# options.add_argument("--headless")
# options.add_argument(f"user-agent={choice(USER_AGENTS)}")
# driver = webdriver.Chrome(options=options)
# driver.get("https://plato.stanford.edu/entries/poincare/")

In [26]:
# prompts = {}
# for prompt_name in [navigator_manifest, reflection, autoreflection, next_move_consideration]:
#     variable_name = [k for k, v in locals().items() if v is prompt_name][0]
#     prompts[variable_name] = prompt_name
# json.dump(prompts, open("prompts.json", "w"))

In [6]:
from reading import read
from searching import search
from session_manager import start_session

readS = start_session(headless=False)
searchS = start_session(headless=False)
root = "https://plato.stanford.edu/entries/poincare/"
#search_concept = read(readS, "https://plato.stanford.edu/entries/poincare/")
#link2content4read = search(searchS, search_concept, readS.current_url)

In [7]:
while True:
    search_concept = read(readS, root)
    link2content4read = search(searchS, search_concept, readS.current_url)
    root = link2content4read

I am reading the following page: https://plato.stanford.edu/entries/poincare/

1. Henri Poincaré was a polymath who made significant contributions in multiple areas of mathematics and the physical sciences.

2. Poincaré's philosophy is strongly influenced by scientific practice and the philosophical doctrines of his time, such as neokantism.

3. Poincaré's work on logicism and geometric conventionalism has had a significant impact on the philosophy of mathematics.

4. Poincaré's hierarchical view of the sciences suggests that the special sciences presuppose physics, which in turn presupposes geometry, which in turn presupposes arithmetic.

5. Poincaré argues that mathematical induction is a necessary synthetic a priori principle in arithmetic.

6. Poincaré's view of geometry challenges the traditional notion of geometry as the a priori science of space and emphasizes the conventional nature of geometrical propositions.

7. Poincaré's structural approach to geometry focuses on the relat

KeyboardInterrupt: 